In [1]:
import numpy as np
import torch
import torch.nn as nn
from lava.lib.dl import slayer
from obd1.dataset import evCIVIL
from obd1.boundingbox import metrics,utils

         Only Prophesee DVS demo will not run properly.
         Please install it from https://github.com/prophesee-ai/prophesee-automotive-dataset-toolbox


In [2]:
def _yolo(x: torch.tensor,
          anchors: torch.tensor,
          clamp_max: float = 5.0) -> torch.tensor:
    # converts raw predictions to bounding box predictions.
    _, _, H, W, _, _ = x.shape
    range_y, range_x = torch.meshgrid(
        torch.arange(H, dtype=x.dtype, device=x.device),
        torch.arange(W, dtype=x.dtype, device=x.device),
        indexing='ij',
    )
    anchor_x, anchor_y = anchors[:, 0], anchors[:, 1]

    x_center = (torch.sigmoid(x[:, :, :, :, 0:1, :])
                + range_x[None, None, :, :, None, None]) / W
    y_center = (torch.sigmoid(x[:, :, :, :, 1:2, :])
                + range_y[None, None, :, :, None, None]) / H
    width = (torch.exp(
        x[:, :, :, :, 2:3, :].clamp(
            max=clamp_max)) * anchor_x[None, :, None, None, None, None]) / W
    height = (torch.exp(
        x[:, :, :, :, 3:4, :].clamp(
            max=clamp_max)) * anchor_y[None, :, None, None, None, None]) / H
    confidence = torch.sigmoid(x[:, :, :, :, 4:5, :])
    classes = torch.softmax(x[:, :, :, :, 5:, :], dim=-2)
    #classes = torch.sigmoid(x[:, :, :, :, 5:, :])

    x = torch.concat([x_center, y_center, width, height,
                      confidence, classes], dim=-2)

    if torch.isnan(x).any() or torch.isinf(x).any():
        print(f'{torch.isnan(x_center).any()=}')
        print(f'{torch.isinf(x_center).any()=}')
        print(f'{torch.isnan(y_center).any()=}')
        print(f'{torch.isinf(y_center).any()=}')
        print(f'{torch.isnan(width).any()=}')
        print(f'{torch.isinf(width).any()=}')
        print(f'{torch.isnan(height).any()=}')
        print(f'{torch.isinf(height).any()=}')
        raise RuntimeError('Ecountered NaN and Inf!')

    return x  # batch, anchor, height, width, predictions, time 



def yolo(x: torch.tensor, anchors: torch.tensor) -> torch.tensor:
        """Evaluates YOLO bounding box prediction from raw network output.

        Parameters
        ----------
        x : torch.tensor
            Raw prediciton tensor.
        anchors : torch.tensor
            Anchors associated with the prediction head.

        Returns
        -------
        torch.tensor
            Output bounding boxes.
        """
        clamp_max = 5.0
        N, _, _, _, P, T = x.shape
        return _yolo(x, anchors, clamp_max).reshape([N, -1, P, T])

def yolo_raw(x: torch.tensor) -> torch.tensor:
        """Transforms raw YOLO prediction to eventual output order i.e.
        NCHWT order to (batch, num_anchors, num_outputs, height, width, time).

        Parameters
        ----------
        x : torch.tensor
            Raw prediction output of the network.

        Returns
        -------
        torch.tensor
            Transformed raw prediction output for a head.
        """
        num_anchors = 3

        N, _, H, W, T = x.shape
        return x.reshape(N,num_anchors,-1, H, W, T).permute(0, 1, 3, 4, 2, 5)

In [3]:
def calculate_detection_scores(fmap_file,ann_file,num_box_stat_file):

    pred_fmaps = np.load(fmap_file)
    all_anns = np.load(ann_file)
    num_boxes_per_sample_arr = np.load(num_box_stat_file)
    num_boxes_per_sample_arr = num_boxes_per_sample_arr.reshape(-1,1)
    print("num boxes per sample arr ",num_boxes_per_sample_arr)

    data_path = "/home/atiye/latest_dataset/"
    test_csv_file = "test_files_event_based.txt"
    param_dict = {"TSteps" : 7, "tbins" : 1 ,"quantized_h" : 260 ,"quantized_w" : 346}
        
    valid_TSteps = 7
    stats = slayer.utils.LearningStats(accuracy_str='AP@0.5')
    ap_stats = metrics.APstats(iou_threshold=0.5)
    anchors = torch.tensor([[0.2800, 0.2200],
        [0.3800, 0.4800],
        [0.9000, 0.7800]])
    anchors = anchors.unsqueeze(0)
    conf_thres = 0.1

    track_id = 0
    total_process_boxes = 0

    for i,(num_boxes,pred_fmap) in enumerate(zip(num_boxes_per_sample_arr,pred_fmaps)):

        num_boxes = num_boxes[0]
        bboxes = all_anns[track_id:(track_id + num_boxes),:]
        print("bboxes ",bboxes)
        print("bboxes shape ",bboxes.shape)
        print("pred_fmap shape ",pred_fmap.shape)
        total_process_boxes += bboxes.shape[0]

        track_id += num_boxes

        bboxes = torch.from_numpy(bboxes)

        pred_fmap = torch.from_numpy(pred_fmap)
        pred_fmap = pred_fmap.unsqueeze(0) #add batch dim at the begining
        pred_fmap = pred_fmap.unsqueeze(-1) #add time dimension
        #now batch,C,H,W,T dimension
        predictions = yolo_raw(pred_fmap)
        predictions = [predictions]
        
        predictions = [prediction/(valid_TSteps + 0.0) for prediction in predictions]
        T = 1 #inputs.shape[-1]  #This means prediction should be done at the last time step
        try:
            predictions = torch.concat([yolo(p, a) for (p, a)
                        in zip(predictions, anchors)],dim=1)
        except RuntimeError:
            print('Runtime error on MAP predictions calculation.'
                            'continuing')
            continue
        predictions = [utils.nms(predictions[..., t],conf_threshold = conf_thres)
                            for t in range(T)]
            
        for t in range(T):
                ap_stats.update(predictions[t],[bboxes]) #bboxes is torch array, predictions[t] is a list. in that list torch.tensor of (75,6) (5x5x3,6)
        
        stats.testing.num_samples += 1
        stats.testing.correct_samples = ap_stats[:] * stats.testing.num_samples

    print("stats.testing.accuracy ",stats.testing.accuracy)
    stats.update()
    print("total process boxes ",total_process_boxes)
    print("all_anns.shape[0] ",all_anns.shape[0])
    assert total_process_boxes == all_anns.shape[0]

In [4]:
#feature core
fmap_file = "float_fmaps.npy"
ann_file = "float_anns.npy"
num_box_stat_file = "float_num_boxes.npy"

calculate_detection_scores(fmap_file,ann_file,num_box_stat_file)

num boxes per sample arr  [[1]
 [1]
 [1]
 [1]
 [1]
 [1]]
bboxes  [[0.6965409  0.31967375 0.16214623 0.2712264  1.         0.        ]]
bboxes shape  (1, 6)
pred_fmap shape  (21, 5, 5)
bboxes  [[0.5692807 0.4086085 0.1572327 0.2702437 1.        0.       ]]
bboxes shape  (1, 6)
pred_fmap shape  (21, 5, 5)
bboxes  [[0.3830582  0.5162146  0.14838837 0.20833333 1.         0.        ]]
bboxes shape  (1, 6)
pred_fmap shape  (21, 5, 5)
bboxes  [[0.3614387 0.5393082 0.1582154 0.2250393 1.        0.       ]]
bboxes shape  (1, 6)
pred_fmap shape  (21, 5, 5)
bboxes  [[0.30001965 0.5663325  0.18671383 0.21619497 1.         0.        ]]
bboxes shape  (1, 6)
pred_fmap shape  (21, 5, 5)
bboxes  [[0.2184552  0.5741942  0.18278302 0.18671383 1.         0.        ]]
bboxes shape  (1, 6)
pred_fmap shape  (21, 5, 5)
stats.testing.accuracy  0.015
total process boxes  6
all_anns.shape[0]  6


In [5]:
#Visualize annotations together with fmaps